In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from huggingface_hub import hf_hub_download
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoProcessor, AutoModel
import requests

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="/data/vdc/tangzichen/qwen2.5",
    # local_dir_use_symlinks=False,
    # torch_dtype="auto",
    # device_map="cuda:1"
)


/home/tangzichen.tzc/anaconda3/envs/humangaussian/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tangzichen.tzc/anaconda3/envs/humangaussian/lib/python3.8/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/tangzichen.tzc/anaconda3/envs/humangaussian/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/tangzichen.tzc/anaconda3/envs/humangaussian/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_downloa

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="/data/vdc/tangzichen/qwen2.5")

In [ ]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

text


In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

model_inputs['input_ids']

In [ ]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

In [ ]:
# 开头一些token是输入, 需要去除
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
generated_ids

In [ ]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

In [ ]:
processor = AutoProcessor.from_pretrained('google/siglip-base-patch16-224', cache_dir="/data/vdc/tangzichen/siglip")

In [ ]:
vision_model = AutoModel.from_pretrained('google/siglip-base-patch16-224', cache_dir="/data/vdc/tangzichen/siglip")

In [ ]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

texts = ["a photo of 2 cats", "a photo of 2 dogs"]

inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")

inputs